In [1]:
import pyomo.environ as pyo
import json
from datetime import timedelta
import polars as pl
from polars import selectors as cs
from polars  import col as c
import os 
import numpy as np
import math
import tqdm
from datetime import timedelta, datetime, timezone
# from optimization_model.optimizaztion_pipeline import first_stage_pipeline
from typing_extensions import Optional
from data_display.input_data_plots import plot_basin_height_volume_table
from baseline_model.optimization_results_processing import *
from data_display.baseline_plots import *
from utility.pyomo_preprocessing import *
from utility.input_data_preprocessing import *
from config import settings
from general_function import pl_to_dict, pl_to_dict_with_tuple, build_non_existing_dirs, generate_log, duckdb_to_dict

from plotly.subplots import make_subplots
import plotly.express as px
import plotly.graph_objs as go
from plotly.graph_objects import Figure

from plotly.subplots import make_subplots

import networkx as nx
from data_display.baseline_plots import *
from baseline_model.baseline_input import BaseLineInput
from baseline_model.first_stage.first_stage_pipeline import BaselineFirstStage
from baseline_model.second_stage.second_stage_pipeline import BaselineSecondStage
COLORS = px.colors.qualitative.Plotly

log = generate_log(name="test")

os.chdir(os.getcwd().replace("/src", ""))
os.environ['GRB_LICENSE_FILE'] = os.environ["HOME"] + "/gurobi_license/gurobi.lic"
volume_factor = 1e-6

In [2]:
input_file_names: dict[str, str] = json.load(open(settings.FILE_NAMES)) # type: ignore
# smallflex_input_schema: SmallflexInputSchema = SmallflexInputSchema().duckdb_to_schema(file_path=input_file_names["duckdb_input"])


In [3]:
output_file_names: dict[str, str] = json.load(open(settings.FILE_NAMES)) # type: ignore
PARALLEL = False
YEARS = [2020, 2021, 2022, 2023]
TURBINE_FACTORS = {0.75, 0.85, 0.95}

SIMULATION_SETTING = [
    {"quantile": 0, "buffer": 0.1, "powered_volume_enabled": True},
    # {"quantile": 0.15, "buffer": 0.3, "powered_volume_enabled": True},
    # {"quantile": 0.25, "buffer": 0.3, "powered_volume_enabled": False},
    # {"quantile": 0.15, "buffer": 0.3, "powered_volume_enabled": True, "global_price": True},
    # {"quantile": 0.25, "buffer": 0.3, "powered_volume_enabled": False, "global_price": True},
]

YEARS = 2020
TURBINE_FACTORS = 0.75
hydro_power_mask = c("name").is_in(["Aegina discrete turbine", "Aegina pump"])
hydro_power_mask = c("name").is_in(["Aegina discrete turbine", "Aegina continuous turbine", "Aegina pump"])
# hydro_power_mask = c("name").is_in(["Aegina discrete turbine"])

REAL_TIMESTEP = timedelta(hours=1)
FIRST_STAGE_TIMESTEP = timedelta(days=1)
SECOND_STAGE_TIME_SIM = timedelta(days=4)
TIME_LIMIT = 20 # in seconds
VOLUME_FACTOR = 1e-6

baseline_input: BaseLineInput = BaseLineInput(
    input_schema_file_name=output_file_names["duckdb_input"],
    real_timestep=REAL_TIMESTEP,
    year=YEARS,
    max_alpha_error=2,
    hydro_power_mask =hydro_power_mask,
    volume_factor=VOLUME_FACTOR
)
first_stage: BaselineFirstStage = BaselineFirstStage(
    nb_state= 1,
    input_instance=baseline_input,
    timestep=FIRST_STAGE_TIMESTEP,
    max_turbined_volume_factor=TURBINE_FACTORS
)
first_stage.solve_model()


second_stage: BaselineSecondStage = BaselineSecondStage(
        input_instance=baseline_input, 
        first_stage=first_stage, 
        timestep=SECOND_STAGE_TIME_SIM, 
        time_limit=TIME_LIMIT,
        model_nb=0,
        nb_state=3,
        is_parallel=PARALLEL,
        **SIMULATION_SETTING[0]
    )
second_stage.solve_model()

Read and validate tables from small_flex_input_data.db file: 100%|████████████████████████████████████████████████████| 15/15 [00:00<00:00, 21.12it/s]
Solving second stage optimization model number 0: 100%|███████████████████████████████████████████████████████████████| 92/92 [00:38<00:00,  2.42it/s]


In [4]:
optimization_summary, combined_results = combine_second_stage_results(
    optimization_results= second_stage.optimization_results,
    powered_volume= second_stage.powered_volume,
    market_price=second_stage.market_price, 
    index=second_stage.index, 
    flow_to_vol_factor= second_stage.real_timestep.total_seconds() * second_stage.volume_factor)

In [5]:
optimization_summary

sim_nb,remaining_volume_0,remaining_volume_1,remaining_volume_2,powered_volume_0,powered_volume_1,powered_volume_2,real_powered_volume_0,real_powered_volume_1,real_powered_volume_2,start_basin_volume_0,start_basin_volume_1
i32,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
0,0.0,0.0,0.0,0.156579,0.156579,0.0,0.195944,0.201507,-0.042668,14.8,0.0
1,-0.039365,-0.044928,-0.042668,0.469737,0.469737,0.0,0.453827,0.447273,-0.068434,14.473219,0.354783
2,-0.023455,-0.022464,-0.111101,0.626316,0.626316,0.0,0.612053,0.615084,-0.034584,13.69286,1.0
3,-0.009192,-0.011232,-0.145686,0.626316,0.626316,0.0,0.591987,0.600536,-0.017381,12.549648,1.0
4,0.025138,0.014548,-0.163066,0.626316,0.626316,0.0,0.633387,0.633591,-0.008871,11.421543,1.0
…,…,…,…,…,…,…,…,…,…,…,…
87,-0.068189,-0.027658,-0.182724,0.156579,0.156579,0.0,0.205257,0.232606,-0.0,15.254001,1.0
88,-0.116867,-0.103685,-0.182724,0.469737,0.469737,0.0,0.410148,0.417895,-0.0,14.87077,1.0
89,-0.057278,-0.051842,-0.182724,0.0,0.0,0.0,0.053333,0.063935,-0.0,14.092844,1.0


In [6]:
combined_results

real_index,timestamp,basin_volume_0,basin_volume_1,volume_0,volume_1,volume_2,hydro_power_0,hydro_power_1,hydro_power_2,spilled_volume_0,spilled_volume_1,market_price,income
u32,"datetime[μs, UTC]",f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
0,2020-01-01 00:00:00 UTC,0.811261,0.0,0.008907,0.008907,-0.0,8.407729,8.407729,0.0,0.0,0.0,35.42,595.603515
1,2020-01-01 01:00:00 UTC,0.810318,0.017813,0.008907,0.007415,-0.0,8.407729,6.999267,0.0,0.0,0.0,34.04,524.454139
2,2020-01-01 02:00:00 UTC,0.809456,0.034134,0.0,0.0,-0.0,0.0,0.0,0.0,0.0,0.0,31.52,0.0
3,2020-01-01 03:00:00 UTC,0.809489,0.034134,0.0,0.0,-0.0,0.0,0.0,0.0,0.0,0.0,28.29,0.0
4,2020-01-01 04:00:00 UTC,0.809521,0.034134,0.0,0.0,-0.008534,0.0,0.0,-11.337381,0.0,0.0,26.92,-305.202299
…,…,…,…,…,…,…,…,…,…,…,…,…,…
8779,2020-12-31 19:00:00 UTC,0.798836,0.529921,0.0,0.0,-0.0,0.0,0.0,0.0,0.0,0.0,58.49,0.0
8780,2020-12-31 20:00:00 UTC,0.798858,0.529921,0.0,0.0,-0.0,0.0,0.0,0.0,0.0,0.0,56.14,0.0
8781,2020-12-31 21:00:00 UTC,0.798881,0.529921,0.0,0.0,-0.0,0.0,0.0,0.0,0.0,0.0,52.98,0.0


In [ ]:
def plot_second_stage_powered_volume(
    simulation_results: pl.DataFrame, fig: Figure, row: int, time_divider: int, **kwargs
    ):
    hydro_name = simulation_results.select(cs.starts_with("volume")).columns
    name = ["Discrete turbine", "Continuous turbine", "Pump"]
    for i, col in enumerate(hydro_name):
        
        data = simulation_results.select(
            c("T"), c(col)
        ).group_by(c("T")//time_divider).agg(pl.all().exclude("T").sum()).sort("T")
        
            
        fig.add_trace(
            go.Bar(
                x=data["T"].to_list(), y=(data[col]).to_list(), showlegend=True,
                marker=dict(color=COLORS[i]), width=1, name= name[i],
                legendgroup="Powered volume"
            ), row=row, col=1
        )
    fig.update_traces(selector=dict(legendgroup="Powered volume"), legendgrouptitle_text="<b>Powered volume [Mm3]<b>")
    return fig

def plot_second_stage_result(
    simulation_results: pl.DataFrame,  time_divider: int) -> Figure:

    fig: Figure = make_subplots(
            rows=3, cols = 1, shared_xaxes=True, vertical_spacing=0.02, x_title="<b>Weeks<b>", 
            row_titles= ["<b>DA price<b>", "<b>Basin water volume<b>", "<b>Powered volume<b>"])

    kwargs: dict = {
            "simulation_results": simulation_results.rename({"real_index": "T"}), 
            "fig": fig,
            "time_divider": time_divider}

    kwargs["fig"] = plot_second_stage_market_price(row=1, **kwargs)
    kwargs["fig"] = plot_basin_volume(row=2, **kwargs)
    fig = plot_second_stage_powered_volume(row=3, **kwargs)
    fig.update_layout(
            barmode='relative',
            margin=dict(t=60, l=65, r= 10, b=60), 
            width=1200,   # Set the width of the figure
            height=800,
            legend_tracegroupgap=180
        )
    
    return fig

fig = plot_second_stage_result(
        simulation_results=combined_results, time_divider=7*24
    )
fig.show()

In [8]:

simulation_results=combined_results
time_divider=7*24

hydro_name = simulation_results.select(cs.starts_with("volume")).columns
hydro_name

['volume_0', 'volume_1', 'volume_2']

In [9]:
combined_results


real_index,timestamp,basin_volume_0,basin_volume_1,volume_0,volume_1,volume_2,hydro_power_0,hydro_power_1,hydro_power_2,spilled_volume_0,spilled_volume_1,market_price,income
u32,"datetime[μs, UTC]",f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
0,2020-01-01 00:00:00 UTC,0.811261,0.0,0.008907,0.008907,-0.0,8.407729,8.407729,0.0,0.0,0.0,35.42,595.603515
1,2020-01-01 01:00:00 UTC,0.810318,0.017813,0.008907,0.007415,-0.0,8.407729,6.999267,0.0,0.0,0.0,34.04,524.454139
2,2020-01-01 02:00:00 UTC,0.809456,0.034134,0.0,0.0,-0.0,0.0,0.0,0.0,0.0,0.0,31.52,0.0
3,2020-01-01 03:00:00 UTC,0.809489,0.034134,0.0,0.0,-0.0,0.0,0.0,0.0,0.0,0.0,28.29,0.0
4,2020-01-01 04:00:00 UTC,0.809521,0.034134,0.0,0.0,-0.008534,0.0,0.0,-11.337381,0.0,0.0,26.92,-305.202299
…,…,…,…,…,…,…,…,…,…,…,…,…,…
8779,2020-12-31 19:00:00 UTC,0.798836,0.529921,0.0,0.0,-0.0,0.0,0.0,0.0,0.0,0.0,58.49,0.0
8780,2020-12-31 20:00:00 UTC,0.798858,0.529921,0.0,0.0,-0.0,0.0,0.0,0.0,0.0,0.0,56.14,0.0
8781,2020-12-31 21:00:00 UTC,0.798881,0.529921,0.0,0.0,-0.0,0.0,0.0,0.0,0.0,0.0,52.98,0.0


In [10]:
for fig_idx, col in enumerate(hydro_name):
    name = col.replace("volume_", "hydro ") 
    data = simulation_results.select(
        c("T"),
        pl.when(c(col) <= 0).then(0).otherwise(c(col)).alias("turbined_volume"),
        pl.when(c(col) >= 0).then(0).otherwise(- c(col)).alias("pumped_volume"),
    ).group_by(c("T")//time_divider).agg(pl.all().exclude("T").sum()).sort("T")
    
    for i, var_name in enumerate(["turbined_volume", "pumped_volume"]):
        factor = 1e-6 if i == 0 else -1e-6
        fig.add_trace(
            go.Bar(
                x=data["T"].to_list(), y=(factor*data[var_name]).to_list(), showlegend=True,
                marker=dict(color=COLORS[i]), width=1, name= var_name.replace("_", " ") + " " + name.replace("_", " "),
                legendgroup=col
            ), row=row +fig_idx, col=1
        )

ColumnNotFoundError: T

In [ ]:
extract_optimization_results(model_instance=second_stage.model_instance, var_name="flow_by_state")

In [ ]:
print(extract_optimization_results(
        model_instance=second_stage.model_instance, var_name="flow"
    ).pivot(on="H", values="flow", index="T").to_pandas().to_string())

In [ ]:
remaining_volume


In [ ]:
print(combined_results.to_pandas().to_string())

In [ ]:
list(find_infeasible_constraints(second_stage.model_instance))

In [ ]:
model_instance = second_stage.model_instance
optimization_results = second_stage.optimization_results
sim_nb = second_stage.sim_nb

for var_name in ["flow", "hydro_power", "basin_volume", "spilled_volume"]:
    data = extract_optimization_results(
            model_instance=model_instance, var_name=var_name
        ).with_columns(
            pl.lit(sim_nb).alias("sim_nb")
        )
    optimization_results[var_name] = pl.concat([optimization_results[var_name], data], how="diagonal_relaxed")
            
        
start_basin_volume = extract_optimization_results(
        model_instance=model_instance, var_name="end_basin_volume"
    ).with_columns(
        pl.lit(sim_nb + 1).alias("sim_nb")
    ).rename({"end_basin_volume": "start_basin_volume"})

remaining_volume = join_pyomo_variables(
        model_instance=model_instance, 
        var_list=["diff_volume_pos", "diff_volume_neg"], 
        index_list=["H"]
    ).select(
        c("H"),
        pl.lit(sim_nb + 1).alias("sim_nb"),
        (c("diff_volume_pos") - c("diff_volume_neg")).alias("remaining_volume"),
    )
optimization_results["start_basin_volume"] = pl.concat([optimization_results["start_basin_volume"], start_basin_volume], how="diagonal_relaxed")
optimization_results["remaining_volume"] = pl.concat([optimization_results["remaining_volume"], remaining_volume], how="diagonal_relaxed")

In [ ]:
optimization_results["remaining_volume"].tail(10)

In [ ]:
start_volume_dict = pl_to_dict(
self.optimization_results["start_basin_volume"].filter(c("sim_nb") == self.sim_nb)[["B", "start_basin_volume"]])

discharge_volume_tot= pl_to_dict(
self.discharge_volume.filter(c("sim_nb") == self.sim_nb).group_by("B").agg(c("discharge_volume").sum()))

self.index["basin_state"], basin_volume = generate_seconde_stage_basin_state(
    index=self.index, water_flow_factor=self.water_flow_factor, 
    basin_volume_table=self.basin_volume_table, start_volume_dict=start_volume_dict, 
    discharge_volume_tot=discharge_volume_tot,
    timestep=self.timestep, volume_factor=self.volume_factor, nb_state=5
)

self.index["hydro_power_state"] = generate_second_stage_hydro_power_state(
    power_performance_table=self.power_performance_table, basin_volume=basin_volume)

In [ ]:
self.index["basin_state"]

In [ ]:
performance_table["power_performance"]

In [ ]:
new_hydro_state.sort("height").interpolate().with_columns(
    (c("power")/ c("flow")).alias("alpha")
)

In [ ]:
start_volume_dict = self.optimization_results["start_volume"].to_dicts()[-1]

In [ ]:
index_b = self.index["water_basin"]["B"].to_list()[0]
timestep = self.timestep

In [ ]:
basin_state

In [ ]:
basin_volume

In [ ]:
arange_float(basin_volume["height"].max(), basin_volume["height"].min(), 0.1)

In [ ]:
water_flow

In [ ]:
performance_table = self.power_performance_table[0]

start_volume = start_volume_dict[performance_table["B"]]

In [ ]:
height_boundary = data.select(pl.concat_list(c("height").min(), c("height").max()))["height"].to_list()[0]

In [ ]:
boundaries = 

In [ ]:
self = second_stage

start_volume_dict = pl_to_dict(
            self.optimization_results["start_basin_volume"].filter(c("sim_nb") == self.sim_nb)[["B", "start_basin_volume"]])

discharge_volume_tot= pl_to_dict(
            self.discharge_volume.filter(c("sim_nb") == self.sim_nb).group_by("B").agg(c("discharge_volume").sum()))

In [ ]:
rated_flow_dict = pl_to_dict(self.index["hydro_power_plant"][["H", "rated_flow"]])


In [ ]:
rated_flow_dict

In [ ]:
state_index: pl.DataFrame = pl.DataFrame()
start_state: int = 0
for performance_table in power_performance_table:
    
    start_volume = start_volume_dict[performance_table["B"]]
    rated_volume = rated_flow_dict[performance_table["H"]] * timestep.total_seconds() * volume_factor
    
    boundaries = (
        start_volume - rated_volume, start_volume + rated_volume + discharge_volume[performance_table["B"]]
    )
    data: pl.DataFrame = filter_data_with_next(
        data=performance_table["power_performance"], col="volume", boundaries=boundaries)
    y_cols = data.select(cs.starts_with(name) for name in ["flow", "electrical"]).columns
    state_name_list = list(set(map(lambda x : x.split("_")[-1], y_cols)))
    
    data = define_state(data=data, x_col="volume", y_cols=y_cols, error_threshold=error_threshold)
    
    data = data\
        .with_row_index(offset=start_state, name="S")\
        .with_columns(
                pl.lit(performance_table["H"]).alias("H"),
                pl.lit(performance_table["B"]).alias("B")
        ).with_columns(
            pl.struct(cs.ends_with(col_name)).name.map_fields(lambda x: "_".join(x.split("_")[:-1])).alias(col_name)
            for col_name in state_name_list
        ).unpivot(
            on=state_name_list, index= ["volume", "S", "H", "B"], value_name="data", variable_name="state"
        ).unnest("data").drop("state")
        
    state_index = pl.concat([state_index, data], how="diagonal")
    start_state = state_index["S"].max() + 1 # type: ignore
    
state_index = state_index.with_row_index(name="S_Q")    
missing_basin: pl.DataFrame = index["water_basin"]\
    .filter(~c("B").is_in(state_index["B"]))\
    .select(
        c("B"),
        pl.struct(
            c("volume_min").fill_null(0.0).alias("min"), 
            c("volume_max").fill_null(0.0).alias("max"),
        ).alias("volume")
    ).with_row_index(offset=start_state, name="S")

index["state"] = pl.concat([state_index, missing_basin], how="diagonal_relaxed")\
    .with_columns(
    pl.concat_list("H", "B", "S", "S").alias("S_BH"),
    pl.concat_list("B", "S").alias("BS"),
    pl.concat_list("H", "S").alias("HS"),
    pl.concat_list("H", "S", "S_Q").alias("HQS")
)

In [ ]:
discharge_volume_tot

In [ ]:
start_volume_dict

In [ ]:


self.data["T"] = {None: self.index["datetime"].filter(c("sim_nb") == self.sim_nb)["T"].to_list()}
self.data["S_B"] = pl_to_dict(
    self.index["state"].unique("S", keep="first")
    .group_by("B", maintain_order=True).agg("S")
    .with_columns(c("S").list.sort())
)
self.data["BS"] = {None: list(map(tuple,self.index["state"]["BS"].to_list()))}


In [ ]:

self.data["HS"] = {None: list(map(tuple,hydropower_state["HS"].to_list()))}
self.data["HQS"] = {None: list(map(tuple,hydropower_state["HQS"].to_list()))}
self.data["S_H"] = pl_to_dict(
    hydropower_state.unique("S", keep="first")
    .group_by("H", maintain_order=True)
    .agg("S")
    .with_columns(c("S").list.sort())
    )
self.data["S_Q"] = pl_to_dict_with_tuple(
    hydropower_state
    .group_by(["HS"], maintain_order=True).agg("S_Q")
    .with_columns(c("S_Q").list.sort())
    )

self.data["B_H"] = pl_to_dict(hydropower_state.group_by("H").agg(c("B").unique()))
self.data["SB_H"] = pl_to_dict_with_tuple(hydropower_state.group_by("HS").agg(c("S").unique()))

self.data["start_basin_volume"] = pl_to_dict(
    self.optimization_results["start_basin_volume"].filter(c("sim_nb") == self.sim_nb)[["B", "start_basin_volume"]])
self.data["remaining_volume"] = pl_to_dict(self.optimization_results["remaining_volume"].filter(c("sim_nb") == self.sim_nb)[["H", "remaining_volume"]])
self.data["min_basin_volume"] = pl_to_dict_with_tuple(
            self.index["state"].select("BS", c("volume").struct.field("min")))
self.data["max_basin_volume"] = pl_to_dict_with_tuple(
    self.index["state"].select("BS", c("volume").struct.field("max")))
self.data["powered_volume_enabled"] = {None: self.powered_volume_enabled}

self.data["discharge_volume"] = pl_to_dict_with_tuple(self.discharge_volume.filter(c("sim_nb") == self.sim_nb)[["TB", "discharge_volume"]])  
self.data["market_price"] = pl_to_dict(self.market_price.filter(c("sim_nb") == self.sim_nb)[["T", "avg"]])
if not self.global_price:
    self.data["neg_unpowered_price"] = {
        None: self.market_price.filter(c("sim_nb") == self.sim_nb)["avg"].quantile(0.5 + self.quantile)}
    self.data["pos_unpowered_price"] = {
        None: self.market_price.filter(c("sim_nb") == self.sim_nb)["avg"].quantile(0.5 - self.quantile)}

self.data["powered_volume"] = pl_to_dict(self.powered_volume.filter(c("sim_nb") == self.sim_nb)[["H", "powered_volume"]])
self.data["volume_buffer"] = pl_to_dict(self.volume_buffer.filter(c("sim_nb") == self.sim_nb)[["H", "volume_buffer"]])

self.data["min_flow"] = pl_to_dict_with_tuple(hydropower_state[["HQS", "flow"]])  
self.data["min_power"] = pl_to_dict_with_tuple(hydropower_state[["HQS", "electrical_power"]])  
self.data["d_flow"] = pl_to_dict_with_tuple(hydropower_state[["HQS", "d_flow"]])  
self.data["d_power"] = pl_to_dict_with_tuple(hydropower_state[["HQS", "d_electrical_power"]])

In [ ]:
second_stage.optimization_results

In [ ]:
self.first_stage_results

In [ ]:
self = second_stage

divisors: int = int(self.timestep / self.first_stage_timestep)

offset = divisors - self.first_stage_results.height%divisors
self.powered_volume = self.first_stage_results.select(
        c("T"), 
        cs.starts_with("powered_volume").name.map(lambda c: c.replace("powered_volume_", "")),
    ).group_by(((c("T") + offset)//divisors).alias("sim_nb"), maintain_order=True)\
    .agg(pl.all().exclude("sim_nb", "T").sum())\
    .unpivot(
        index="sim_nb", variable_name="H", value_name="powered_volume"
    ).with_columns(
        c("H").cast(pl.UInt32).alias("H")
    )

In [ ]:
self.index["hydro_power_plant"]\
            .select(
                c("H").cast(pl.Utf8), 
                c("rated_flow") * self.real_timestep.total_seconds() * self.volume_factor * self.buffer
            )

In [ ]:
self.process_timeseries()

In [ ]:
self.discharge_volume 

In [ ]:
plot_first_stage_result(
    simulation_results=first_stage.optimization_results, 
    time_divider=7
).show()

In [ ]:
first_stage.water_flow_factor

In [ ]:
first_stage.optimization_results

In [ ]:
extract_optimization_results(
            model_instance=first_stage.model_instance, var_name="spilled_volume"
        )


In [ ]:

turbined_power = extract_optimization_results(
        model_instance=model_instance, var_name="turbined_power"
    )

turbined_power = pivot_result_table(
    df = turbined_power, on="H", index=["T"],
    values="turbined_power")

simulation_results: pl.DataFrame = market_price\
    .join(basin_volume, on = "T", how="inner")\
    .join(turbined_volume, on = "T", how="inner")\
    .join(pumped_volume, on = "T", how="inner")\
    .join(pumped_power, on = "T", how="inner")\
    .join(turbined_power, on = "T", how="inner")\
    .with_columns(
        ((
            pl.sum_horizontal(cs.starts_with("turbined_power")) -
            pl.sum_horizontal(cs.starts_with("pumped_power"))
        ) * c("T").replace_strict(nb_hours_mapping, default=None) * c("market_price")).alias("income")
    )
    
hydro_name = list(map(str, list(model_instance.H))) # type: ignore

simulation_results = simulation_results.with_columns(
    pl.struct(cs.ends_with(hydro) & ~cs.starts_with("basin_volume"))
    .pipe(remove_suffix).alias("hydro_" + hydro) 
    for hydro in hydro_name
).select(    
    ~(cs.starts_with("turbined") | cs.starts_with("pumped")) # type: ignore
)
